In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd, datetime, numpy as np
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import warnings
warnings.filterwarnings('ignore') #Tiingo API is returning
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Define the Stocks, the holdings  in a dataframe**

In [ ]:
data={'Stocks':['GOOGL','TSLA','AAPL'],'Quantity':[100,50,300]}# Define the stocks
df=pd.DataFrame(data)
df

Sourcing the Historical data from Tiingo API


In [1]:
pip install tiingo

In [ ]:
from tiingo import TiingoClient
def SourceHistoricPrices():
    if info==1: 
        print('[INFO] Fetching stock prices for portfolio holdings')
        # Setup Tiingo
        config={}
        config['Session']=True
        config['Session']='private key'
        client=TiingoClient(config)
        
        
        # Create a list of tickers for the Tiingo API
        Tickers=[]
        for ticker in data:
            while i<=len(data):
                Tickers.append(data[ticker][i])
                i=i+1
        if info==1:
            print('[INFO] Portfolio Holdings determined as ',Tickers)
            global HistData
            HistData=client.get_dataframe(Ticker,metric_name='close',startDate=dateforNoOfScenarios(today),endDate=today)
        if info==1:
            print('[INFO] Fetching Stock Prices completed',len(HistData),'days.')
            return(HistData)
            

Define Historical simulations 
By calculating the involved business days.

In [ ]:
ScenarioNo=500 # Number of iternations to run
today=datetime.date.today()-datetime.timedelta(days=1)
def is_business_day(date):
    return bool(len(pd.bdate_range(date,date)))
def dateforNoOfScenarios(date):
    i=0 # Business day variable count
    w=0 # Normal day variable count
    while i <ScenarioNo:
        if (is_business_day(today - datetime.timedelta(days = w)) == True):
            # Increase the count 
            i=i+1
            w=w+1
        else:
            w=w+1
            continue
            print("System going back these business days: ",i)
            print("System going back these days: ",w)
            # Add an extra day as percentage difference
            # 4% is the arbitary percent of to days from 500 days
            return (today-datetime.timedelta(days=w*1.04+1))
        
        

In [ ]:
def ValuePortfolio():
    HistData['PortValue']=0
    i=0
    if info==1:
        print('[INFO] Calculating the portfolio value for each day')
        while i<=len(data):
            stock=data['Stocks'][i]
            quantity=data['Quantity'][i]
            HistData['PortValue']=HistData['PortValue']+HistData[stock]*quantity
            i=i+1

 Define the Portfolio Valuation change

In [ ]:
HistData['DollarChange']=HistData.loc[HistData.index.max()]['PortValue']*HistData['Perc_Change'] 
# Cacluate the change in money $ from the current valuation
if info==1:
    print('[INFO] Picking',round(HistData.loc[HistData.index.max()]['PortValue'],2),'value from',
          HistData.index.max().strftime('%Y-%m-%d'),
          'as the latest valuation to base the monetary returns'
         )

Confidence Interval from the nth value

In [ ]:
Percentile = 99 #Define your confidence interval
ValueLocForPercentile = round(len(HistData) * (1 - (Percentile / 100)))
if info == 1: print('[INFO] Picking the', ValueLocForPercentile, 'th highest value')

Sort the data and select the value:

In [ ]:
global SortedHistData
SortedHistData = HistData.sort_values(by=['DollarChange'])
if info == 1: print('[INFO] Sorting the results by highest max loss')

Calculate our portfolio’s VaR value

In [ ]:
VarDaysHorizon = 1 #Define your time period
def CalculateVaR():
    if info == 1: print('[INFO] Calculating Daily % Changes')
    #calculating percentage change
    HistData['Perc_Change'] = HistData['PortValue'].pct_change()
    #calculate money change based on current valuation
    HistData['DollarChange'] = HistData.loc[HistData.index.max()]['PortValue'] * HistData['Perc_Change'] 
    if info == 1: print('[INFO] Picking', round(HistData.loc[HistData.index.max()]['PortValue'],2),' value from ', HistData.index.max().strftime('%Y-%m-%d'), ' as the latest valuation to base the monetary returns')
    ValueLocForPercentile = round(len(HistData) * (1 - (Percentile / 100)))
    if info == 1: print('[INFO] Picking the', ValueLocForPercentile, 'th highest value')
    global SortedHistData
    SortedHistData = HistData.sort_values(by=['DollarChange'])
    if info == 1: print('[INFO] Sorting the results by highest max loss')
    VaR_Result = SortedHistData.iloc[ValueLocForPercentile + 1,len(SortedHistData.columns)-1] * np.sqrt(VarDaysHorizon)
    print('The portfolio\'s VaR is:', round(VaR_Result,2))

In [ ]:
def CalculateES():
    ValueLocForPercentile = round(len(HistData) * (1 - (Percentile / 100)))
    ES_Result = round(SortedHistData['DollarChange'].head(ValueLocForPercentile).mean(axis=0),2) * np.sqrt(VarDaysHorizon)
    print('The portfolios\'s Expected Shortfall is', ES_Result)

In [ ]:
SourceHistoricPrices()
ValuePortfolio()
CalculateVaR()
CalculateES()

Use a histogram to see how our portfolio returns fit with the assumption

In [ ]:
import scipy
import scipy.stats, matplotlib.pyplot as plt
def plotme():
    data1 = HistData['Perc_Change']
    num_bins = 50
    # the histogram of the data
    n, bins, patches = plt.hist(data1, num_bins, normed=1, facecolor='green', alpha=0.5)
    # add a 'best fit' line
    sigma = HistData['Perc_Change'].std()
    data2 = scipy.stats.norm.pdf(bins, 0, sigma)
    plt.plot(bins, data2, 'r--')
    plt.xlabel('Percentage Change')
    plt.ylabel('Probability/Frequency')
    # Tweak spacing to prevent clipping of ylabel
    plt.subplots_adjust(left=0.15)
    plt.show()
plotme()